In [1]:
!pip install -U langgraph langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling langchain-0.3.27:
      Successfully uninstalled langchain-0.3.27


The API Key should be named "OPENAI_API_KEY", while creating the key (https://openrouter.ai/settings/keys)

In [29]:
import os
os.environ["OPENAI_API_KEY"] = "INSERT_API_KEY_HERE"

#INSERT KEY HERE


In [30]:
import os
from dotenv import load_dotenv

load_dotenv()

False

In [31]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="openai/gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
)

In [32]:
from typing_extensions import TypedDict

class CounterState(TypedDict):
    count: int

In [33]:
def increment(state: CounterState) -> dict:
    state["count"] += 1
    return state

In [34]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(CounterState)

builder.add_node("increment", increment)

# Define the execution order: START -> increment -> END
builder.add_edge(START, "increment")
builder.add_edge("increment", END)

graph = builder.compile()

In [35]:
initial_state: CounterState = {"count": 0}

result = graph.invoke(initial_state)

print(result)

# Output
# {'count': 1}

{'count': 1}


In [37]:
from typing import Literal

def should_continue(state: CounterState) -> Literal["increment", END]:

    if state["count"] <= 3: # keep looping
      #print("count:", count)
      return "increment"


    return END # stop the graph

In [38]:
builder = StateGraph(CounterState)

builder.add_node("increment", increment)

builder.add_edge(START, "increment")

builder.add_conditional_edges(
    "increment",
    should_continue,
    ["increment", END],
)

graph = builder.compile()

result = graph.invoke({"count": 0})

print(result)

# Output
# {'count': 3}

{'count': 4}


In [39]:
from typing_extensions import TypedDict

class AgentState(TypedDict):
    user_input: str
    response: str

In [40]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="openai/gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
)

In [41]:
from langchain_core.messages import SystemMessage, HumanMessage

def llm_node(state: AgentState) -> dict:
    messages = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=state["user_input"]),
    ]

    reply = llm.invoke(messages)

    return {"response": reply.content}

In [42]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(AgentState)

builder.add_node("llm", llm_node)

# define the flow: START -> llm -> END
builder.add_edge(START, "llm")
builder.add_edge("llm", END)

graph = builder.compile()

In [48]:
initial_state: AgentState = {
    "user_input": "Tell me about Ashlyn Viereck at Creighton University?",
    "response": "",
}

result = graph.invoke(initial_state)

print("User:", initial_state["user_input"])
print("Assistant:", result["response"])

User: Tell me about Ashlyn Viereck at Creighton University?
Assistant: As of my last knowledge update in October 2021, I don’t have specific information on an individual named Ashlyn Viereck at Creighton University. It's possible that she is a student, faculty member, or involved in some capacity at the university, but there are no publicly available details in my training data regarding her.

For the most accurate and current information, I recommend visiting Creighton University's official website or contacting the university directly. You might also check social media platforms or news releases that could feature updates about students or staff.


In [ ]:
state2: AgentState = {
    "user_input": "Can you summarise that in two lines?",
    "response": "",
}
result2 = graph.invoke(state2)

print("\nTurn 2 - User:", state2["user_input"])
print("Turn 2 - Assistant:", result2["response"])


Turn 2 - User: Can you summarise that in two lines?
Turn 2 - Assistant: Of course! However, I need the content you want summarized. Please provide the text or main ideas you'd like to condense into two lines.


In [ ]:
from typing_extensions import TypedDict, Annotated
from operator import add
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver

class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]

def node_a(state: State):
    # overwrite foo, append "a" to bar
    return {"foo": "a", "bar": ["a"]}

def node_b(state: State):
    # overwrite foo, append "b" to bar
    return {"foo": "b", "bar": ["b"]}

In [ ]:
builder = StateGraph(State)
builder.add_node("node_a", node_a)
builder.add_node("node_b", node_b)

builder.add_edge(START, "node_a")
builder.add_edge("node_a", "node_b")
builder.add_edge("node_b", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
config = {"configurable": {"thread_id": "1"}}

final_state = graph.invoke({"foo": "", "bar": []}, config=config)
print("Final state:", final_state)

# Output
# Final state: {'foo': 'b', 'bar': ['a', 'b']}

Final state: {'foo': 'b', 'bar': ['a', 'b']}


In [ ]:
history = list(graph.get_state_history(config))

for i, snap in enumerate(history[::-1]):
    print(f"\nCheckpoint {i}:")
    print("  created_at:", snap.created_at)
    print("  node:", snap.metadata)
    print("  values:", snap.values)


Checkpoint 0:
  created_at: 2025-11-18T20:29:25.831759+00:00
  node: {'source': 'input', 'step': -1, 'parents': {}}
  values: {'bar': []}

Checkpoint 1:
  created_at: 2025-11-18T20:29:25.833885+00:00
  node: {'source': 'loop', 'step': 0, 'parents': {}}
  values: {'foo': '', 'bar': []}

Checkpoint 2:
  created_at: 2025-11-18T20:29:25.835782+00:00
  node: {'source': 'loop', 'step': 1, 'parents': {}}
  values: {'foo': 'a', 'bar': ['a']}

Checkpoint 3:
  created_at: 2025-11-18T20:29:25.836613+00:00
  node: {'source': 'loop', 'step': 2, 'parents': {}}
  values: {'foo': 'b', 'bar': ['a', 'b']}


In [ ]:
from typing_extensions import TypedDict, Annotated
from langchain_core.messages import AnyMessage
import operator

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
from langchain_core.messages import SystemMessage

def chat_llm_node(state: MessagesState) -> dict:

    # build prompt from system + existing conversation
    history = [SystemMessage(content="You are a helpful assistant.")]
    history.extend(state["messages"])

    reply = llm.invoke(history)

    return {"messages": [reply]}

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage

checkpointer = InMemorySaver()

builder = StateGraph(MessagesState)
builder.add_node("chat_llm", chat_llm_node)
builder.add_edge(START, "chat_llm")
builder.add_edge("chat_llm", END)

graph = builder.compile(checkpointer=checkpointer)

In [ ]:
config = {"configurable": {"thread_id": "user_123"}}


In [ ]:
config = {"configurable": {"thread_id": "user_123"}}

# Turn 1
state1 = {"messages": [HumanMessage(content="What is GIL in python?")]}
result1 = graph.invoke(state1, config=config)

for m in result1["messages"]:
    print(type(m).__name__, ":", m.content)

# Turn 2
state2 = {"messages": [HumanMessage(content="Summarise it in two lines")],}
result2 = graph.invoke(state2, config=config)

for m in result2["messages"][-2:]:
    print(type(m).__name__, ":", m.content)

HumanMessage : What is GIL in python?
AIMessage : The Global Interpreter Lock (GIL) is a mechanism used in the CPython implementation of Python to ensure that only one thread executes Python bytecode at a time. This means that even though Python supports multi-threading, the GIL can prevent true parallel execution of threads in a single process. 

Here are some key points about the GIL:

1. **Purpose**: The GIL simplifies memory management and protects access to Python objects, making it easier to maintain internal state while allowing threads to share the same interpreter.

2. **Impact on Multi-threading**: Because of the GIL, CPU-bound multi-threaded programs might not see a performance improvement because the GIL allows only one thread to execute at a time. However, I/O-bound thread programs can benefit from multi-threading, as the GIL is released while waiting for I/O operations to complete.

3. **Workarounds**: To achieve true parallelism in CPU-bound tasks, Python developers can 